# Consolidate data sources into one single Data Frame

## Import data from Yahoo Finance

In [31]:
# import libraries

import yfinance as yf
import pandas as pd

In [32]:
# Get gold, oil and DXY data from Yahoo Finance

gold_prices = yf.download("GC=F", start="2016-01-01", end="2025-02-28")
gold_prices.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,GC=F,GC=F,GC=F,GC=F,GC=F
Date,,,,,
2016-01-04,1075.099976,1082.500000,1063.199951,1063.400024,143
2016-01-05,1078.400024,1081.500000,1075.300049,1075.599976,82
2016-01-06,1091.900024,1093.699951,1081.599976,1081.599976,52
2016-01-07,1107.699951,1109.400024,1091.599976,1091.599976,122
2016-01-08,1097.800049,1111.099976,1093.000000,1111.099976,98


We have to prepare the dataset in order to make it useful for our analysis

In [33]:
# Getting rid of extra headers
gold_prices = yf.download("GC=F", start="2016-01-01", end="2025-02-28", group_by='ticker')
gold_prices.columns = gold_prices.columns.droplevel(0)

# Move 'Date' from the index to a normal column
gold_prices.reset_index(inplace=True)
gold_prices["Date"] = pd.to_datetime(gold_prices["Date"])

gold_prices.head()

[*********************100%***********************]  1 of 1 completed

Price,Date,Open,High,Low,Close,Volume
0,2016-01-04,1063.400024,1082.500000,1063.199951,1075.099976,143
1,2016-01-05,1075.599976,1081.500000,1075.300049,1078.400024,82
2,2016-01-06,1081.599976,1093.699951,1081.599976,1091.900024,52
3,2016-01-07,1091.599976,1109.400024,1091.599976,1107.699951,122
4,2016-01-08,1111.099976,1111.099976,1093.000000,1097.800049,98


In [44]:
# Repeat the process for Oil and DXY dataframes from Yahoo Finance and keeping the close column

dxy = yf.download("DX-Y.NYB", start="2016-01-01", end="2025-02-28")
dxy.columns = dxy.columns.droplevel(1)
dxy.reset_index(inplace=True)
dxy["Date"] = pd.to_datetime(dxy["Date"])
dxy = dxy.sort_values("Date")[["Date", "Close"]].rename(columns={"Close": "dxy_close"})

oil = yf.download("BZ=F", start="2016-01-01", end="2025-02-28")
oil.columns = oil.columns.droplevel(1)
oil.reset_index(inplace=True)
oil["Date"] = pd.to_datetime(oil["Date"])
oil = oil.sort_values("Date")[["Date", "Close"]].rename(columns={"Close": "oil_close"})

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [45]:
# Load external data - This csv files were downloaded from the Federal Reserve Economic Data (https://fred.stlouisfed.org/)

fedfunds = pd.read_csv("../data/external/FEDFUNDS.csv")
gs10 = pd.read_csv("../data/external/GS10.csv")
m2real = pd.read_csv("../data/external/M2REAL.csv")

# Convert date columns to datetime objects

fedfunds["observation_date"] = pd.to_datetime(fedfunds["observation_date"])
gs10["observation_date"] = pd.to_datetime(gs10["observation_date"])
m2real["observation_date"] = pd.to_datetime(m2real["observation_date"])

# Get a sneak peak of data
fedfunds.head()

,observation_date,FEDFUNDS
0,2001-01-01,5.98
1,2001-02-01,5.49
2,2001-03-01,5.31
3,2001-04-01,4.80
4,2001-05-01,4.21


In [46]:
# Load processed BLS data
bls_data = pd.read_csv("../data/processed/bls_data.csv")

# Have a sneak peak
bls_data.head()

,year,period,periodName,latest,value,footnotes,seriesID
0,2025,M02,February,True,319.082,[{}],CUUR0000SA0
1,2025,M01,January,NaN,317.671,[{}],CUUR0000SA0
2,2024,M12,December,NaN,315.605,[{}],CUUR0000SA0
3,2024,M11,November,NaN,315.493,[{}],CUUR0000SA0
4,2024,M10,October,NaN,315.664,[{}],CUUR0000SA0


* We can check the entire dataset in the /data/processed/ directory. We'll proceed to prepare the data set for further analysis

In [47]:
# create date column
bls_data["month"] = bls_data["period"].str.extract(r'M(\d{2})')
bls_data["Date"] = pd.to_datetime(bls_data["year"].astype(str) + "-" + bls_data["month"] + "-01")

# Check the result
bls_data.head()

,year,period,periodName,latest,value,footnotes,seriesID,month,Date
0,2025,M02,February,True,319.082,[{}],CUUR0000SA0,02,2025-02-01
1,2025,M01,January,NaN,317.671,[{}],CUUR0000SA0,01,2025-01-01
2,2024,M12,December,NaN,315.605,[{}],CUUR0000SA0,12,2024-12-01
3,2024,M11,November,NaN,315.493,[{}],CUUR0000SA0,11,2024-11-01
4,2024,M10,October,NaN,315.664,[{}],CUUR0000SA0,10,2024-10-01


* We'll have to pivot the dataset so each seriesID becomes a column

In [48]:
# Pivot the dataset and keep relevant columns
bls_data = bls_data.pivot(index="Date", columns="seriesID", values="value")
bls_data = bls_data.apply(pd.to_numeric, errors='coerce').reset_index()
bls_data.head()

seriesID,Date,CUUR0000SA0,LNS14000000,WPUFD49104
0,2016-01-01,236.916,4.8,109.7
1,2016-02-01,237.111,4.9,109.9
2,2016-03-01,238.132,5.0,109.9
3,2016-04-01,239.261,5.1,110.0
4,2016-05-01,240.229,4.8,110.0


In [ ]:
# Consilidate the data in a single dataset
consolidated = pd.merge_asof(gold_prices.sort_values("Date"), dxy.sort_values("Date"), left_on="Date", right_on="Date", direction="backward", suffixes=("", "_dxy"))
consolidated = pd.merge_asof(consolidated.sort_values("Date"), oil.sort_values("Date"), left_on="Date", right_on="Date", direction="backward", suffixes=("", "_oil"))
consolidated = pd.merge_asof(consolidated.sort_values("Date"), fedfunds.sort_values("observation_date"), left_on="Date", right_on="observation_date", direction="backward")
consolidated = pd.merge_asof(consolidated.sort_values("Date"), gs10.sort_values("observation_date"), left_on="Date", right_on="observation_date", direction="backward")
consolidated = pd.merge_asof(consolidated.sort_values("Date"), m2real.sort_values("observation_date"), left_on="Date", right_on="observation_date", direction="backward")
consolidated = pd.merge_asof(consolidated.sort_values("Date"), bls_data.sort_values("Date"), left_on="Date", right_on="Date", direction="backward")

# Remove extra columns, rename them and save to csv
consolidated.drop(columns=["observation_date_x", "observation_date_y", "observation_date",], inplace=True)
consolidated.columns = consolidated.columns.str.lower()
consolidated = consolidated.rename(columns={"cuur0000sa0": "cpi", "lns14000000": "nfp", "wpufd49104": "ppi"})

consolidated.to_csv("../data/processed/consolidated.csv")
consolidated.head()

,date,open,high,low,close,volume,dxy_close,oil_close,fedfunds,gs10,m2real,cpi,nfp,ppi
0,2016-01-04,1063.400024,1082.500000,1063.199951,1075.099976,143,98.870003,37.220001,0.34,2.09,5255.9,236.916,4.8,109.7
1,2016-01-05,1075.599976,1081.500000,1075.300049,1078.400024,82,99.400002,36.419998,0.34,2.09,5255.9,236.916,4.8,109.7
2,2016-01-06,1081.599976,1093.699951,1081.599976,1091.900024,52,99.180000,34.230000,0.34,2.09,5255.9,236.916,4.8,109.7
3,2016-01-07,1091.599976,1109.400024,1091.599976,1107.699951,122,98.220001,33.750000,0.34,2.09,5255.9,236.916,4.8,109.7
4,2016-01-08,1111.099976,1111.099976,1093.000000,1097.800049,98,98.540001,33.549999,0.34,2.09,5255.9,236.916,4.8,109.7
